In [1]:
import pandas as pd
import pdblp
import datetime
from dateutil.parser import parse
import numpy as np

In [2]:
action_id = "5154510"

In [3]:
deals = ["5154510", "5948716"]
deals = [id + " Action" for id in deals]

In [4]:
date = datetime.datetime.today().date()
con = pdblp.BCon(debug=False, timeout=5000)
con.start()
df_tmp1 = con.ref(deals, 'CA057')#, start_date=date.strftime('%y%m%d'), end_date=date.strftime('%y%m%d'))
con.stop()

In [5]:
df_tmp1

,ticker,field,value
0,5154510 Action,CA057,2000-01-10
1,5948716 Action,CA057,1999-11-14


In [6]:
moneyness = ['80', '90.0', '95.0', '97.5', '100.0', '102.5', '105.0', '110.0', '120']
tenors = ['30DAY', '60DAY', '3MTH', '6MTH', '12MTH', '18MTH', '24MTH']

df = pd.DataFrame(index=tenors, columns=moneyness)

vol_fields = []

for i in tenors:
    for j in moneyness:
        vol_fields.append("{}_IMPVOL_{}%MNY_DF".format(i, j))

In [7]:
for i, j in df.iteritems():
    print(j.index, i)

Index(['30DAY', '60DAY', '3MTH', '6MTH', '12MTH', '18MTH', '24MTH'], dtype='object') 80
Index(['30DAY', '60DAY', '3MTH', '6MTH', '12MTH', '18MTH', '24MTH'], dtype='object') 90.0
Index(['30DAY', '60DAY', '3MTH', '6MTH', '12MTH', '18MTH', '24MTH'], dtype='object') 95.0
Index(['30DAY', '60DAY', '3MTH', '6MTH', '12MTH', '18MTH', '24MTH'], dtype='object') 97.5
Index(['30DAY', '60DAY', '3MTH', '6MTH', '12MTH', '18MTH', '24MTH'], dtype='object') 100.0
Index(['30DAY', '60DAY', '3MTH', '6MTH', '12MTH', '18MTH', '24MTH'], dtype='object') 102.5
Index(['30DAY', '60DAY', '3MTH', '6MTH', '12MTH', '18MTH', '24MTH'], dtype='object') 105.0
Index(['30DAY', '60DAY', '3MTH', '6MTH', '12MTH', '18MTH', '24MTH'], dtype='object') 110.0
Index(['30DAY', '60DAY', '3MTH', '6MTH', '12MTH', '18MTH', '24MTH'], dtype='object') 120


In [8]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

date = parse("10/23/2015")
start_date = parse("20060101")
end_date = parse("20061231")

In [9]:
def get_vol_surface(ticker, date):
    
    def batch(iterable, n=1):
        l = len(iterable)
        for ndx in range(0, l, n):
            yield iterable[ndx:min(ndx + n, l)]

    if not isinstance(date, datetime.date):
        date = parse(date)

    vols_list = []

    for fields in batch(vol_fields, 25):
        try:
            con = pdblp.BCon(debug=False)
            con.start()
            df_tmp = con.bdh(ticker, fields,
                              start_date=date.strftime('%y%m%d'),
                              end_date=date.strftime('%y%m%d'))
            con.stop()
            vols_list.extend(list(df_tmp.values[0]))
        except Exception as e:
            raise(e)
            con.stop()
            
    df = pd.DataFrame(np.array(vols_list).reshape(len(tenors), len(moneyness)), 
                      index=tenors, columns=moneyness)
    return df

In [10]:
bd = get_vol_surface('RAD US Equity', '10/23/2015')

In [11]:
ad = get_vol_surface('RAD US Equity', '11/24/2015')

In [12]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
from matplotlib import cm
# %matplotlib inline

In [13]:
ad-bd

,80,90.0,95.0,97.5,100.0,102.5,105.0,110.0,120
30DAY,-33.1323,-28.8652,-25.6152,-24.2040,-24.6538,-25.1039,-25.5541,-26.9889,-26.9144
60DAY,-9.6980,-20.3177,-22.2869,-23.3948,-25.6381,-26.5690,-26.1318,-25.1438,-20.1135
3MTH,-11.9720,-18.1391,-20.6495,-21.9430,-23.6871,-24.8227,-25.3130,-26.1624,-23.3603
6MTH,-15.5708,-18.2896,-20.3045,-21.3431,-22.6684,-23.7280,-24.5160,-26.0922,-26.1777
12MTH,-15.6296,-18.8377,-20.3509,-21.1079,-21.9080,-22.6472,-23.3267,-24.6833,-26.9138
18MTH,-15.8027,-19.2986,-20.7894,-21.5364,-22.2844,-23.0333,-23.7833,-25.2862,-28.0228
24MTH,-15.3873,-19.1874,-21.0256,-21.9456,-22.8663,-23.7876,-24.7094,-26.5550,-29.4884


In [14]:
X, Y = np.meshgrid([i for i in range(len(ad.index))], np.asfarray(ad.columns))
Z = ad.values.T

In [ ]:
# X = np.array([ad.index.values] * len(ad.columns))
# Y = np.array([ad.columns.values] * len(ad.index))
# Z = ad.values
%matplotlib
fig = plt.figure(figsize=(15, 8))
threedee = fig.add_subplot(111, projection='3d')#.gca(projection='3d')
threedee.plot_surface(X, Y, Z, cmap=cm.coolwarm, rstride=1, cstride=1, linewidth=0, antialiased=False)
threedee.set_xticklabels(tenors);
threedee.view_init(30, 20);
threedee.invert_xaxis();
threedee.set_zlim(0, 50);

for angle in range(0, 360):
    threedee.view_init(30, angle)
    plt.draw()
    plt.pause(.001)

In [98]:
X = np.arange(-5, 5, 0.25)
Y = np.arange(-5, 5, 0.25)
X, Y = np.meshgrid(X, Y)
R = np.sqrt(X**2 + Y**2)
Z = np.sin(R)
